## Skip Gram Model

P(word|context) -> CBOW
skip gram -> P(Wcontext | Wword) it provides the word embeddings , given the words it predicts context for the word embeddings. Why do we require the embeddings? These embeddings captures the semantic relationships among the words because one hot encoding doesn't captures the semantic relationship 
    - because one-hot encoding is a sparse matrix
    - the euclidean distance of root(2) doesn't gurantees the far away 
Skip gram model thus captures the:
    - semantic relationship
    - syntactic relationship (explain it later)

## Question
1. Word Representation using Word2Vec: Implement the skip-gram model with negative
sampling loss function for word embedding generation. Your implementation should include: [20
 Marks]
 (b) Implement the skip-gram model from scratch with negative sampling loss. [4]
 (c) Derive and implement the gradients for backpropagation. [4]
 (d) Train your model on the text8 dataset with appropriate hyperparameters (specify your choices
 and justify them). [3]
 (e) Evaluate the quality of your embeddings through: [4]
 • Visualization using SVD to project the embeddings to 2D space.
 • Word similarity analysis for semantically related words (e.g., “king”- “man” + “woman”
 ≈ “queen”).
 (f) Discuss the impact of key hyperparameters (e.g., embedding dimension, context window size,
 number of negative samples) on the quality of the learned representations. [2]

In [ ]:
import torch.nn.functional as F
import torch
import numpy as np
import torch.nn as nn

class SkipGram:
    def __init__(self,input,hidden,output=1):
        self.hidden=hidden
        self.output=output
        self.input=input
        self.W1 = torch.tensor([[np.random.normal() for _ in range(self.output)] for _ in range(self.input)]) #how do i initialize a tensor of the size input X hidden
        self.W2 = torch.tensor([[np.random.normal() for _ in range(self.output)] for _ in range(self.input)])
        self.b1 = torch.tensor([np.random.normal() for _ in range(self.hidden)])
        self.b2 = torch.tensor([np.random.normal() for _ in range(self.output)])
        
        self.eta =0.01
        
    def forward(self,input_tensor):
        f = torch.tanh(torch.add(torch.matmul(self.W1.T,input_tensor),self.b1))
        g = torch.add(torch.matmul(self.W2.T,f),self.b2)
        output= F.softmax(g)
        # how do we find the log of this 
        return output , f
    
    def cross_entropy(self,X,y):
        m = y.shape[0]
        p = F.log_softmax(X,dim=1)
        loss = -p[torch.arange(m), y].mean()
        return loss
    
    def delta_cross_entropy(self,X,y):
        m = y.shape[0]
        grad = F.softmax(X).clone()
        grad[torch.arange(m), y] -= 1
        grad = grad/m
        return grad
    
    def gradient_descent(self,X,y):
        ##self.W1-=
        #cross_entropy_loss = self.cross_entropy()
        n_epoch = 10
        
        for _ in range(n_epoch):
            prediction,f = self.forward(X)
            loss = self.cross_entropy(prediction,y)
            
            # calculate the gradients of loss wrt crossentropy function
            d_g = prediction.clone()
            d_g[torch.arange(y.shape[0]), y] -= 1
            d_g /= y.shape[0]

            # calculate the deltaW1, deltaW2 and deltab1, deltab2 
            deltaW2 = torch.matmul(f.T,d_g)
            db2 = torch.sum(d_g,dim=0)
            
            d_f = torch.matmul(d_g, self.W2.T) * (1 - f**2) 
            deltaW1 = torch.matmul(X.T,d_f)
            db1 = torch.sum(d_f,dim=0)
            self.W1 -= self.eta*deltaW1
            self.W2 -= self.eta*deltaW2
            self.b1 -= self.eta*db1
            self.b2 -= self.eta*db2
            print(f'Epoch {_+1}, Loss: {loss.item():.4f}')
